In [16]:
import requests
import kfp

#working HOST = 'http://10.100.59.230:31973'
#NOT WORKING HOST = 'http://ml-pipeline.kubeflow.svc.cluster.local:8888'

#WORKING
HOST='http://hua-kubeflow.ddns.net/'
USERNAME = "user@example.com"
PASSWORD = "2LZHseTdrLFFvx"
NAMESPACE = "kubeflow-user-example-com"

HOST = 'http://huanew-kubeflow.ddns.net/pipeline'
USERNAME = "gfatouros@innov-acts.com"
PASSWORD = "Gxpnn8l4"
NAMESPACE = "innovacts"

session = requests.Session()
response = session.get(HOST)

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
}

data = {"login": USERNAME, "password": PASSWORD}
response = session.post(HOST, headers=headers, data=data)
# session_cookie = session.cookies.get_dict()["oauth2_proxy_kubeflow"]
session_cookie = "gIDXBTNHJ0fFAnEhSi0cDnEU1U-T1Yk94ktc8Y6n9wlZ0nAQE3rlc0zeZSBlqngVXUcSEgcCwVmv-R3IHNHHX5NmxSyqjFWgHu_Ki5U-ShWQl4fWbJ716QAOGCWItJr2KJtFa2CFgn41yfEEguNhsmZQdvx_tTuOaGpXhif1Z0WchC1_bPX-wLaCPpwo2jiuSiGKVjSH2q4E1Gw2GBdwIlNmNXC9otLuA47hb5Qdwu-Cz6Lr85Sx4N_eyzzFCUE2hTGWGeXM7GeZGKDOWo_Bzw4dhamIQhPqcffgXx2tLwEE7AKfaQ4a0lg2qVJO-BmAbgJmIOhktNEVapdY9Fh8s3deEKgl3ktZ8Ruq06I5kzyKqZeYm5dJPhbUDBiEHdHGQVlXz2yMwv6F4bABQsqQIhWGFQ7gwMxaxmIxizGVbgJBmPyh-5ATfuapLWba4zbnQ9mz2xQ-t5Vdi2qywWuhDKtlcZFIekb2QmJIFsAPEfiAmtMRDKpdTaxLUx7TU2GqbO8gtV-BM8Bg83UNHr_3ZIJsTDP-SgKhOpLbEu-YIWC396fRGOeoRpqQlZxx4k7SImfzERn8877KVaA-Yl6nCv8bmxewjJRJcOMwJ5IS8F3oBzZYU1J6b0UUmpi4vVVCWMTbx06TPwe0o2gza3GVG31QPXCoFv99DSEl3ngJ-Hewym4DZJsTjd00BaCcatJ2aJY3lZbQJfr1YUwlb7j3I9f8T1QwXyzywK_RGBM4xHQdEogpesRUQ0RrfNy6i7vCkkbypcaSyEJtmbi4Iy9x32f36u5uAia4o43Dyz_vaPadJ1cF29Z-FMv2i_aGkdPwfgol32FJ_RyPE5GI7554MB_0HIVPDiA59laRxdK7TFdH0U310v869XMlrYzeASaUKY1LhBHfDXvIpta4ipMciOTN8VTJ3qfvrUG5QtfTb_pBnyQOR3Rp4IpKexRvxO0WDBV15bSm5UQFd4TieDcfYvp0JhuHNHMn1hAKmG3vI7igYGw3ssQRUo1xyoy9A_iHYWCuxyOVmUhrhaJeix9MuOrKGAjZRUAfErSsncTxm2_5JKHVPLJ2S_23Ne8tkeRWCalvcNMlkAI40UpFfiswOcvC8nWZZY4JjOrYKN6dKZFLRGYmaB9ifGBhtGiBprw0gRW9Gc2PVFiD0c62jQJ4c0iGIAsxo_m4ZE4DOMv6IpIGFpr2zZRcjDl07H2x6uwisU-YJLn7147poqu7zXIQVztcvDmN7N9LMveRI1VN2D5_YNLnCmcGbMDFshh-bLft793epOM60meFpXS0oWGPYtUE0l_aLcODtGe_1VoaChuGMop6xAVcmmjgixpe8ooItEsk1Uv-fVHCrktyVuls6JJZZKtzoEA059Oa1rIQtSq97w7Lp3Fio_sJGeZe-pCsoFc2xE9gSmDFiZ8SreByEyi2DkRnXNIbqPKNedW7mbTqYytiIhYMKuGuImbxyVi_DUHWrRQW0VqHRWwYcSO11hs1kYIilODZr-X8MrYbnx0Retinn_YmmjaXhH_r9uwoyguTm3yDvEtbJz57p8apdxLo1BpgrNKqFf2JlRuvmdP-C9HAWlgg6fLDvLqIhCo0d-Q4-SFKW1_UM-QYIS0RCIr8Q6ythWUzn33rdfR6YwHNv03_h7dw7UjcAZZaNfiyAX4oxkDy8xKdGyGJGwpUTG7ysxDIU_YqScrlwlvvZPtfXgRGpoZSilvsTAEwV1S1f8_wUVc5LwO4jZeQGl6oE7zoFvlFdrdAWhoTSTB3G4ixQLg5zL7_slHt3itEsUoL6-XHRSGC1qqUxjOQJ0j72L6RH_UyZsxc_X7lvu9ly3SVqXU7UJB1pEXEOlDaDWdNVXJ7r6YR9hONR-oV4RE3Y-YR2pz3zew_wAmewlqzzbSQbpWOvxf2PYy4kG-W2OO4okZm4aWe3KIkE2GbVNMRlL9TUzNoOpkKBeguBAE_WAm55LblnTnd|1743027071|-e0YgHmNwsEVxnKVdQIvPr1wJYKmLEjS07pZwNflOds="
client = kfp.Client(
    host=HOST,
    cookies=f"oauth2_proxy_kubeflow={session_cookie}",
    namespace=NAMESPACE,
)

 

# test that we have a valid session
print(client.list_pipelines())
print(client.get_user_namespace())


#print(client.get_recurring_run("2fc815d5-6861-4241-b3d6-5bcdb5445b0b"))

{'next_page_token': None,
 'pipelines': [{'created_at': datetime.datetime(2024, 11, 24, 8, 1, 21, tzinfo=tzutc()),
                'description': '[source '
                               'code](https://github.com/kubeflow/pipelines/tree/4467df5f9c8242d389f20214f75fecfc4119e8cd/samples/tutorials/Data%20passing%20in%20python%20components) '
                               'Shows how to pass data between python '
                               'components.',
                'display_name': '[Tutorial] Data passing in python components',
                'error': None,
                'namespace': None,
                'pipeline_id': '6311f629-33cb-42aa-aeb9-de44e57e4f85'},
               {'created_at': datetime.datetime(2024, 11, 24, 8, 1, 23, tzinfo=tzutc()),
                'description': '[source '
                               'code](https://github.com/kubeflow/pipelines/tree/4467df5f9c8242d389f20214f75fecfc4119e8cd/samples/tutorials/DSL%20-%20Control%20structures) '
                 

In [21]:
client.list_pipelines()

{'next_page_token': None,
 'pipelines': [{'created_at': datetime.datetime(2024, 11, 24, 8, 1, 21, tzinfo=tzutc()),
                'description': '[source '
                               'code](https://github.com/kubeflow/pipelines/tree/4467df5f9c8242d389f20214f75fecfc4119e8cd/samples/tutorials/Data%20passing%20in%20python%20components) '
                               'Shows how to pass data between python '
                               'components.',
                'display_name': '[Tutorial] Data passing in python components',
                'error': None,
                'namespace': None,
                'pipeline_id': '6311f629-33cb-42aa-aeb9-de44e57e4f85'},
               {'created_at': datetime.datetime(2024, 11, 24, 8, 1, 23, tzinfo=tzutc()),
                'description': '[source '
                               'code](https://github.com/kubeflow/pipelines/tree/4467df5f9c8242d389f20214f75fecfc4119e8cd/samples/tutorials/DSL%20-%20Control%20structures) '
                 

In [1]:
import kfp
import requests
from urllib.parse import urljoin
import re

# Kubeflow settings
HOST = 'http://huanew-kubeflow.ddns.net'
PIPELINE_HOST = 'http://huanew-kubeflow.ddns.net/pipeline'
USERNAME = "gfatouros@innov-acts.com"
PASSWORD = "Gxpnn8l4"
NAMESPACE = "innovacts"

def authenticate_kubeflow_dex():
    """Go through the full Dex authentication flow for Kubeflow"""
    session = requests.Session()
    
    # Step 1: Visit the login page directly with the state parameter
    auth_url = f"{HOST}/dex/auth/local/login?back=&state=xadqsv6m75q3o5bshnpb5jrst"
    response = session.get(auth_url)
    
    # Get the login form to extract any CSRF tokens or hidden fields
    form_html = response.text
    
    # Find form action URL - it might be relative or absolute
    action_match = re.search(r'<form[^>]*action="([^"]*)"', form_html)
    if action_match:
        form_action = action_match.group(1)
        # Make absolute URL if it's relative
        if not form_action.startswith('http'):
            if form_action.startswith('/'):
                form_action = HOST + form_action
            else:
                form_action = urljoin(response.url, form_action)
    else:
        # If no form action found, use the same URL without trailing slash
        form_action = auth_url.rstrip('/')
    
    print(f"Form action URL: {form_action}")
    
    # Step 2: Submit login credentials
    login_data = {
        "login": USERNAME,
        "password": PASSWORD,
        # Add any hidden fields like CSRF tokens here if needed
    }
    
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Referer": auth_url
    }
    
    response = session.post(form_action, data=login_data, headers=headers, allow_redirects=True)
    print(f"Login response status: {response.status_code}")
    print(f"Login response URL: {response.url}")
    
    # Print cookies to debug
    print("Cookies after auth flow:", session.cookies.get_dict())
    
    return session

# Get authenticated session
auth_session = authenticate_kubeflow_dex()

Form action URL: http://huanew-kubeflow.ddns.net/dex/auth/local/login?back=&state=xadqsv6m75q3o5bshnpb5jrst
Login response status: 400
Login response URL: http://huanew-kubeflow.ddns.net/dex/auth/local/login?back=&state=xadqsv6m75q3o5bshnpb5jrst
Cookies after auth flow: {}


In [4]:
session_cookie = session.cookies.get_dict()

In [6]:
session

In [3]:
# Initialize the client manager with Dex credentials
kfp_client_manager = KFPClientManager(
    api_url=f"{HOST}/pipeline",
    skip_tls_verify=True,
    dex_username=USERNAME,
    dex_password=PASSWORD,
    dex_auth_type="local",  # or 'ldap' depending on your Dex configuration
)

# Create an authenticated KFP client
client = kfp_client_manager.create_kfp_client()

# Test the client
print(client.list_pipelines())
print(client.get_user_namespace())

NameError: name 'KFPClientManager' is not defined

In [2]:
#  Kubeflow connection settings
KUBEFLOW_HOST = 'http://hua-kubeflow.ddns.net/'
KUBEFLOW_USERNAME = "user@example.com"
KUBEFLOW_PASSWORD = "2LZHseTdrLFFvx"
KUBEFLOW_NAMESPACE = "kubeflow-user-example-com"

def get_kubeflow_client() -> kfp.Client:
    """Initialize and return a Kubeflow Pipelines client."""
    session = requests.Session()
    response = session.get(KUBEFLOW_HOST)
    
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
    }
    
    data = {"login": KUBEFLOW_USERNAME, "password": KUBEFLOW_PASSWORD}
    session.post(response.url, headers=headers, data=data)
    session_cookie = session.cookies.get_dict()["authservice_session"]
    
    return kfp.Client(
        host=f"{KUBEFLOW_HOST}/pipeline",
        cookies=f"authservice_session={session_cookie}",
        namespace=KUBEFLOW_NAMESPACE,
    )

In [3]:
kf_client = get_kubeflow_client()
pipelines = kf_client.list_pipelines()
print(pipelines)
        


{'next_page_token': 'eyJTb3J0QnlGaWVsZE5hbWUiOiJDcmVhdGVkQXRJblNlYyIsIlNvcnRCeUZpZWxkVmFsdWUiOjE3MzIyNzIwMTMsIlNvcnRCeUZpZWxkUHJlZml4IjoicGlwZWxpbmVzLiIsIktleUZpZWxkTmFtZSI6IlVVSUQiLCJLZXlGaWVsZFZhbHVlIjoiZWFiZDMwYTItZGMzMi00NGNhLWI1ZGYtMjViMjNjZjRkNzZhIiwiS2V5RmllbGRQcmVmaXgiOiJwaXBlbGluZXMuIiwiSXNEZXNjIjpmYWxzZSwiTW9kZWxOYW1lIjoicGlwZWxpbmVzIiwiRmlsdGVyIjpudWxsfQ==',
 'pipelines': [{'created_at': datetime.datetime(2024, 2, 20, 21, 26, 40, tzinfo=tzutc()),
                'description': '[source '
                               'code](https://github.com/kubeflow/pipelines/tree/63ca91850a9f42a357f3417110a3011ddbf43290/samples/tutorials/Data%20passing%20in%20python%20components) '
                               'Shows how to pass data between python '
                               'components.',
                'display_name': '[Tutorial] Data passing in python components',
                'error': None,
                'namespace': None,
                'pipeline_id': 'c5013489-db5f-4

In [2]:
import re
from urllib.parse import urlsplit, urlencode

import kfp
import requests
import urllib3


class KFPClientManager:
    """
    A class that creates `kfp.Client` instances with Dex authentication.
    """

    def __init__(
        self,
        api_url: str,
        dex_username: str,
        dex_password: str,
        dex_auth_type: str = "local",
        skip_tls_verify: bool = False,
    ):
        """
        Initialize the KfpClient

        :param api_url: the Kubeflow Pipelines API URL
        :param skip_tls_verify: if True, skip TLS verification
        :param dex_username: the Dex username
        :param dex_password: the Dex password
        :param dex_auth_type: the auth type to use if Dex has multiple enabled, one of: ['ldap', 'local']
        """
        self._api_url = api_url
        self._skip_tls_verify = skip_tls_verify
        self._dex_username = dex_username
        self._dex_password = dex_password
        self._dex_auth_type = dex_auth_type
        self._client = None

        # disable SSL verification, if requested
        if self._skip_tls_verify:
            urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

        # ensure `dex_default_auth_type` is valid
        if self._dex_auth_type not in ["ldap", "local"]:
            raise ValueError(
                f"Invalid `dex_auth_type` '{self._dex_auth_type}', must be one of: ['ldap', 'local']"
            )

    def _get_session_cookies(self) -> str:
        """
        Get the session cookies by authenticating against Dex
        :return: a string of session cookies in the form "key1=value1; key2=value2"
        """

        # use a persistent session (for cookies)
        s = requests.Session()

        # GET the api_url, which should redirect to Dex
        resp = s.get(
            self._api_url, allow_redirects=True, verify=not self._skip_tls_verify
        )
        if resp.status_code == 200:
            pass
        elif resp.status_code == 403:
            # if we get 403, we might be at the oauth2-proxy sign-in page
            # the default path to start the sign-in flow is `/oauth2/start?rd=<url>`
            url_obj = urlsplit(resp.url)
            url_obj = url_obj._replace(
                path="/oauth2/start", query=urlencode({"rd": url_obj.path})
            )
            resp = s.get(
                url_obj.geturl(), allow_redirects=True, verify=not self._skip_tls_verify
            )
        else:
            raise RuntimeError(
                f"HTTP status code '{resp.status_code}' for GET against: {self._api_url}"
            )

        # if we were NOT redirected, then the endpoint is unsecured
        if len(resp.history) == 0:
            # no cookies are needed
            return ""

        # if we are at `../auth` path, we need to select an auth type
        url_obj = urlsplit(resp.url)
        if re.search(r"/auth$", url_obj.path):
            url_obj = url_obj._replace(
                path=re.sub(r"/auth$", f"/auth/{self._dex_auth_type}", url_obj.path)
            )

        # if we are at `../auth/xxxx/login` path, then we are at the login page
        if re.search(r"/auth/.*/login$", url_obj.path):
            dex_login_url = url_obj.geturl()
        else:
            # otherwise, we need to follow a redirect to the login page
            resp = s.get(
                url_obj.geturl(), allow_redirects=True, verify=not self._skip_tls_verify
            )
            if resp.status_code != 200:
                raise RuntimeError(
                    f"HTTP status code '{resp.status_code}' for GET against: {url_obj.geturl()}"
                )
            dex_login_url = resp.url

        # attempt Dex login
        resp = s.post(
            dex_login_url,
            data={"login": self._dex_username, "password": self._dex_password},
            allow_redirects=True,
            verify=not self._skip_tls_verify,
        )
        if resp.status_code != 200:
            raise RuntimeError(
                f"HTTP status code '{resp.status_code}' for POST against: {dex_login_url}"
            )

        # if we were NOT redirected, then the login credentials were probably invalid
        if len(resp.history) == 0:
            raise RuntimeError(
                f"Login credentials are probably invalid - "
                f"No redirect after POST to: {dex_login_url}"
            )

        # if we are at `../approval` path, we need to approve the login
        url_obj = urlsplit(resp.url)
        if re.search(r"/approval$", url_obj.path):
            dex_approval_url = url_obj.geturl()

            # approve the login
            resp = s.post(
                dex_approval_url,
                data={"approval": "approve"},
                allow_redirects=True,
                verify=not self._skip_tls_verify,
            )
            if resp.status_code != 200:
                raise RuntimeError(
                    f"HTTP status code '{resp.status_code}' for POST against: {url_obj.geturl()}"
                )

        return "; ".join([f"{c.name}={c.value}" for c in s.cookies])

    def _create_kfp_client(self) -> kfp.Client:
        try:
            session_cookies = self._get_session_cookies()
        except Exception as ex:
            raise RuntimeError(f"Failed to get Dex session cookies") from ex

        # monkey patch the kfp.Client to support disabling SSL verification
        # kfp only added support in v2: https://github.com/kubeflow/pipelines/pull/7174
        original_load_config = kfp.Client._load_config

        def patched_load_config(client_self, *args, **kwargs):
            config = original_load_config(client_self, *args, **kwargs)
            config.verify_ssl = not self._skip_tls_verify
            return config

        patched_kfp_client = kfp.Client
        patched_kfp_client._load_config = patched_load_config

        return patched_kfp_client(
            host=self._api_url,
            cookies=session_cookies,
        )

    def create_kfp_client(self) -> kfp.Client:
        """Get a newly authenticated Kubeflow Pipelines client."""
        return self._create_kfp_client()
            

In [1]:
from utils.helper_functions import KFPClientManager

ModuleNotFoundError: No module named 'utils'

In [3]:
HOST = 'http://huanew-kubeflow.ddns.net/pipeline'
USERNAME = "gfatouros@innov-acts.com"
PASSWORD = "Gxpnn8l4"
NAMESPACE = "innovacts"

kfp_client_manager = KFPClientManager(
    api_url=HOST,
    skip_tls_verify=True,

    dex_username=USERNAME,
    dex_password=PASSWORD,

    # can be 'ldap' or 'local' depending on your Dex configuration
    dex_auth_type="ldap",
)

# get a newly authenticated KFP client
# TIP: long-lived sessions might need to get a new client when their session expires
kfp_client = kfp_client_manager.create_kfp_client()

# test the client by listing experiments
experiments = kfp_client.list_experiments(namespace=NAMESPACE)
print(experiments)

{'experiments': [{'created_at': datetime.datetime(2025, 3, 6, 14, 23, 53, tzinfo=tzutc()),
                  'description': 'Test-simple-ccn',
                  'display_name': 'med-minst-exp',
                  'experiment_id': '0d73987e-3e9e-4cd9-b6ad-66a29c1de208',
                  'namespace': 'innovacts',
                  'storage_state': 'AVAILABLE'},
                 {'created_at': datetime.datetime(2025, 3, 29, 0, 21, 36, tzinfo=tzutc()),
                  'description': None,
                  'display_name': 'Default',
                  'experiment_id': 'b6fa3bc7-e8af-4ea3-acf4-1ab328fa8f0e',
                  'namespace': 'innovacts',
                  'storage_state': 'AVAILABLE'}],
 'next_page_token': None,
 'total_size': 2}


/Users/george/Library/Caches/pypoetry/virtualenvs/humaine-swarm-H_nw7hre-py3.11/lib/python3.11/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


In [2]:
# Fix MinIO connection settings
MINIO_ACCESS_KEY="JTnvpo8gGkioE6QhJAq7"
MINIO_SECRET_KEY="Ei281BuHUHihXf93PNhAUdIF8QV1Cm2mVhUbCPqn"
MINIO_SECURE=True
MINIO_ENDPOINT="minio.humaine-horizon.eu:9000"  # Remove the path component
MINIO_PATH="/api/v1/"  # Store the path separately if needed

from minio import Minio
from minio.error import S3Error

def get_minio_client() -> Minio:
    """Initialize and return a MinIO client."""
    client = Minio(
        endpoint=MINIO_ENDPOINT,  # Use just the domain part
        access_key=MINIO_ACCESS_KEY,
        secret_key=MINIO_SECRET_KEY,
        secure=MINIO_SECURE
    )
    
    # If you need to handle the path component, you might need to
    # set it in the client configuration or handle it when making requests
    return client



In [3]:
client = get_minio_client()
        
      